In [5]:
import pandas as pd
import numpy as np
import re

In [6]:
# načtení dat – filmy
movie_col_names = ['movie_id', 'title', 'genre']
movies = pd.read_csv('./movies.dat', sep="::", header=None, names=movie_col_names, engine="python", encoding="latin")

# načtení dat - hodnocení
ratings_col_names = ['movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('./ratings.dat', sep="::", header=None, names=ratings_col_names, engine="python")

In [7]:
# spojení tabulek hodnocení a filmů
data = pd.merge(movies, ratings)

# odstranění datumu z názvu filmu
title = [] 
for values in data['title']:
    title.append(re.sub(r'(\([1-3][0-9]{3})\)', '', values))

data['title'] = title
data.head()

,movie_id,title,genre,rating,timestamp
0,1,Toy Story,Animation|Children's|Comedy,5,978824268
1,1,Toy Story,Animation|Children's|Comedy,4,978237008
2,1,Toy Story,Animation|Children's|Comedy,4,978233496
3,1,Toy Story,Animation|Children's|Comedy,5,978225952
4,1,Toy Story,Animation|Children's|Comedy,5,978226474


In [19]:
############## Výpočet hodnocení filmů ############## 

mean_ratings = data.pivot_table(index='title', values='rating', aggfunc=[np.mean, len])

# odstranění prvního řádku atributů pro usnadnění práce
mean_ratings.columns = mean_ratings.columns.droplevel(1)
mean_ratings.columns.name = None
mean_ratings = mean_ratings.reset_index()

# pomocné parametry pro následující výpočet
m = (mean_ratings['mean'] * mean_ratings['len'] / mean_ratings['len'].sum(axis=0)).sum(axis=0)
C = mean_ratings['len'].quantile(0.25)
# záleží na nastavení parametru C
# s větší velikostí C je brán větší ohled na počet hodnocení

# výpočet hodnocení pomocí bayesova průměru
mean_ratings['score'] = (C * m + mean_ratings['len'] * mean_ratings['mean']) / (C + mean_ratings['len'])

# setřízení
mean_ratings = mean_ratings.sort_values(by='score', ascending=False)

mean_ratings


C:\Users\Martin\AppData\Local\Temp\ipykernel_45788\320859421.py:3: FutureWarning: The provided callable <function mean at 0x00000161BBABE0C0> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  mean_ratings = data.pivot_table(index='title', values='rating', aggfunc=[np.mean, len])


33.0


,title,mean,len,score
2934,"Shawshank Redemption, The",4.554558,2227,4.540350
2905,Seven Samurai (The Magnificent Seven) (Shichin...,4.560510,628,4.511636
1345,"Godfather, The",4.524966,2223,4.511167
3464,"Usual Suspects, The",4.517106,1783,4.500106
2866,Schindler's List,4.510417,2304,4.497301
...,...,...,...,...
247,Baby Geniuses,1.701220,164,2.016201
3078,Speed 2: Cruise Control,1.871935,367,2.012979
2579,Police Academy 6: City Under Siege,1.657718,149,2.006547
1786,Kazaam,1.466667,120,1.922821
